# NOTEBOOK 4.1 Neo4j

Read in/replace the values for the following variables with your own:
- URI
- neo4j_password

In [ ]:
from neo4j import GraphDatabase

URI = "<Your AuraDB neo4j cluster URI>" ### !!!! REMEMBER TO DELETE AND REPLACE

neo4j_user = "neo4j"
neo4j_password = "<Your AuraDB neo4j cluster password>"  ###  !!!! REMEMBER TO DELETE AND REPLACE

AUTH = (neo4j_user, neo4j_password)
driver = GraphDatabase.driver(URI, auth=AUTH)

In [ ]:
driver.verify_connectivity()

# Create a node representing a person named Alice

In [ ]:
summary = driver.execute_query(
    "CREATE (:Person {name: $name})",
    name="Alice",
    database_="neo4j",
).summary

In [ ]:
print("Created {nodes_created} nodes in {time} ms.".format(
    nodes_created = summary.counters.nodes_created,
    time=summary.result_available_after
))

Created 1 nodes in 39 ms.


# Retrieve all Person nodes

In [ ]:
records, summary, keys = driver.execute_query(
    "MATCH (p:Person) RETURN p.name AS name",
    database_="neo4j",
)

# Loop through results and do something with them
for record in records:
    print(record.data())  # obtain record as dict

# Summary information
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after
))

{'name': 'Alice'}
The query `MATCH (p:Person) RETURN p.name AS name` returned 1 records in 27 ms.


# Update the database

In [ ]:
records, summary, keys = driver.execute_query("""
    MATCH (p:Person {name: $name})
    SET p.age = $age
    """, name="Alice", age=42,
    database_="neo4j",
)

print(f"Query counters: {summary.counters}.")

Query counters: {'_contains_updates': True, 'properties_set': 1}.


# Create a relationship :KNOWS between Alice and Bob

In [ ]:
records, summary, keys = driver.execute_query("""
    MATCH (p:Person {name: $name})
    CREATE (p)-[:KNOWS]->(:Person {name: $friend})
    """, name="Alice", friend="Bob",
    database_="neo4j",
)

print(f"Query counters: {summary.counters}.")

Query counters: {'_contains_updates': True, 'labels_added': 1, 'relationships_created': 1, 'nodes_created': 1, 'properties_set': 1}.


# Remove the Alice node

In [ ]:
records, summary, keys = driver.execute_query("""
    MATCH (p:Person {name: $name})
    DETACH DELETE p
    """, name="Alice",
    database_="neo4j",
)

print(f"Query counters: {summary.counters}.")

Query counters: {'_contains_updates': True, 'nodes_deleted': 1, 'relationships_deleted': 1}.


# Query parameters

## (a) Pass query parameters as keyword arguments

In [ ]:
driver.execute_query(
    "MERGE (:Person {name: $name})",
    name="Alice", age=42,
    database_="neo4j",
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7f0ca19cd690>, keys=[])

## (b) Pass query parameters in a dictionary

In [ ]:
parameters = {
    "name": "Hobbes",
    "age": 23
}

driver.execute_query(
    "MERGE (:Person {name: $name})",
    parameters_=parameters,
    database_="neo4j",
)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x7f0ca16295d0>, keys=[])

In [ ]:
records, summary, keys = driver.execute_query(
    "MATCH (p:Person) RETURN p.name AS name",
    database_="neo4j",
)

print(records)

[<Record name='Alice'>, <Record name='Bob'>, <Record name='Hobbes'>]


In [ ]:
people = [{"name": "Garfield", "age": 42, "friends": ["Odie", "Peter", "Anna"]},
          {"name": "Odie", "age": 19},
          {"name": "Peter", "age": 50},
          {"name": "Anna", "age": 30}]

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    try:
        # Create some nodes
        for person in people:
            records, summary, keys = driver.execute_query(
                "MERGE (p:Person {name: $person.name, age: $person.age})",
                person=person,
                database_="neo4j",
            )

        # Create some relationships
        for person in people:
            if person.get("friends"):
                records, summary, keys = driver.execute_query("""
                    MATCH (p:Person {name: $person.name})
                    UNWIND $person.friends AS friend_name
                    MATCH (friend:Person {name: friend_name})
                    MERGE (p)-[:KNOWS]->(friend)
                    """, person=person,
                    database_="neo4j",
                )

        # Retrieve Garfield's friends who are under 40
        records, summary, keys = driver.execute_query("""
            MATCH (p:Person {name: $name})-[:KNOWS]-(friend:Person)
            WHERE friend.age < $age
            RETURN friend
            """, name="Garfield", age=40,
            routing_="r",
            database_="neo4j",
        )
        # Loop through results and do something with them
        for record in records:
            print(record)
        # Summary information
        print("The query `{query}` returned {records_count} records in {time} ms.".format(
            query=summary.query, records_count=len(records),
            time=summary.result_available_after
        ))

    except Exception as e:
        print(e)
        # further logging/processing


<Record friend=<Node element_id='4:b225ba6d-ca62-48b1-9a46-0e48d0e4641d:4' labels=frozenset({'Person'}) properties={'name': 'Odie', 'age': 19}>>
<Record friend=<Node element_id='4:b225ba6d-ca62-48b1-9a46-0e48d0e4641d:6' labels=frozenset({'Person'}) properties={'name': 'Anna', 'age': 30}>>
The query `
            MATCH (p:Person {name: $name})-[:KNOWS]-(friend:Person)
            WHERE friend.age < $age
            RETURN friend
            ` returned 2 records in 54 ms.
